In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
dataset_path = 'dataset'

In [5]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [6]:
train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)
val_gen=datagen.flow_from_directory(
    dataset_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [7]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from tensorflow.keras.models import Sequential

In [8]:
import tensorflow as tf

In [9]:
# CNN Model

In [10]:
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))

model.add(Conv2D(filters=36,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 220, 220, 36)        │           5,220 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 110, 110, 36)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 108, 108, 64)        │          20,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       5,537,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,638,245 (21.51 MB)

 Trainable params: 5,638,245 (21.51 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
import keras

In [13]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [17]:
es = EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=5,verbose=1,mode='auto')
mc = ModelCheckpoint(monitor='val_accuracy',filepath='best_model.h5',verbose=1,save_best_only=True,mode='auto')

In [18]:
hs = model.fit(train_gen,steps_per_epoch=6, epochs=30, verbose=1, validation_data=val_gen,
                        validation_steps=1,callbacks=[es,mc])

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8647 - loss: 0.4379
Epoch 1: val_accuracy improved from -inf to 0.71875, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8598 - loss: 0.4398 - val_accuracy: 0.7188 - val_loss: 0.6027
Epoch 2/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.7500 - loss: 0.5325
Epoch 2: val_accuracy improved from 0.71875 to 0.75000, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - accuracy: 0.7500 - loss: 0.5325 - val_accuracy: 0.7500 - val_loss: 0.5952
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8089 - loss: 0.4365
Epoch 3: val_accuracy did not improve from 0.75000
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.8117 - loss: 0.4353 - val_accuracy: 0.7188 - val_loss: 0.5685
Epoch 4/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 4s 949ms/step - accuracy: 0.9091 - loss: 0.3664
Epoch 4: val_accuracy did not improve from 0.75000
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 138ms/step - accuracy: 0.9091 - loss: 0.3664 - val_accuracy: 0.7188 - val_loss: 0.6161
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8370 - loss: 0.3798
Epoch 5: val_accuracy improved from 0.75000 to 0.78125, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.8377 - loss: 0.3786 - val_accuracy: 0.7812 - val_loss: 0.4959
Epoch 6/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.8125 - loss: 0.4939
Epoch 6: val_accuracy did not improve from 0.78125
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.8125 - loss: 0.4939 - val_accuracy: 0.6875 - val_loss: 0.6006
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9106 - loss: 0.2693
Epoch 7: val_accuracy did not improve from 0.78125
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.9083 - loss: 0.2722 - val_accuracy: 0.6875 - val_loss: 0.6040
Epoch 8/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.7812 - loss: 0.6232
Epoch 8: val_accuracy improved from 0.78125 to 0.81250, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - accuracy: 0.7812 - loss: 0.6232 - val_accuracy: 0.8125 - val_loss: 0.4416
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8723 - loss: 0.3467
Epoch 9: val_accuracy did not improve from 0.81250
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.8755 - loss: 0.3422 - val_accuracy: 0.7500 - val_loss: 0.4057
Epoch 10/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9062 - loss: 0.2830
Epoch 10: val_accuracy improved from 0.81250 to 0.87500, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 240ms/step - accuracy: 0.9062 - loss: 0.2830 - val_accuracy: 0.8750 - val_loss: 0.2846
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9086 - loss: 0.1935
Epoch 11: val_accuracy did not improve from 0.87500
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9083 - loss: 0.1955 - val_accuracy: 0.8438 - val_loss: 0.3026
Epoch 12/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.9062 - loss: 0.2392
Epoch 12: val_accuracy improved from 0.87500 to 0.90625, saving model to best_model.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 240ms/step - accuracy: 0.9062 - loss: 0.2392 - val_accuracy: 0.9062 - val_loss: 0.3947
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9369 - loss: 0.1814
Epoch 13: val_accuracy did not improve from 0.90625
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9384 - loss: 0.1798 - val_accuracy: 0.8750 - val_loss: 0.3546
Epoch 14/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9375 - loss: 0.1674
Epoch 14: val_accuracy did not improve from 0.90625
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 133ms/step - accuracy: 0.9375 - loss: 0.1674 - val_accuracy: 0.8125 - val_loss: 0.4031
Epoch 15/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9262 - loss: 0.1514
Epoch 15: val_accuracy did not improve from 0.90625
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.9267 - loss: 0.1507 - val_accuracy: 0.9062 - val_loss: 0.3732
Epoch 16/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.0630
Epoch 16: val_accuracy did not improve from 0.90625
6/6 ━━

In [29]:
print('Num GPUs Available',len(tf.config.list_physical_devices('GPU')))

Num GPUs Available 0


In [19]:
from keras.preprocessing.image import load_img, img_to_array

In [24]:
img_path = 'abc.png'

In [25]:
img = load_img(img_path, target_size=(224,224))

In [26]:
img_array = img_to_array(img)

In [27]:
img_array = img_array/255

In [28]:
img_array = np.expand_dims(img_array,axis=0)

In [30]:
prediction=model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 978ms/step


In [31]:
prediction

array([[0.99999934]], dtype=float32)

In [32]:
if prediction[0][0] >= 0.5:
    print('Tumor Detected')
else:
    print('No Tumor Detected')

Tumor Detected
